In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import gensim
from gensim import corpora, models
import string
from sklearn.feature_extraction.text import CountVectorizer
import ast
from nrclex import NRCLex
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn import svm 
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
from matplotlib.colors import ListedColormap
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.stats import loguniform
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score, fbeta_score

In [2]:
emotion_df = pd.read_csv("df_emotions.csv")
emotion_df.head()


,Unnamed: 0,text,class,stemmed_processed_text,lemmatized_processed_text,clean_lemmatized_processed_text,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,anticipation
0,0,ex wife threatening suicide recently i left my...,suicide,"['ex', 'wife', 'threaten', 'suicid', 'recent',...","['ex', 'wife', 'threatening', 'suicide', 'rece...",ex wife threatening suicide recently left wife...,0.125000,0.125000,0.0,0.078125,0.078125,0.078125,0.171875,0.093750,0.062500,0.078125,0.109375
1,1,am i weird i do not get affected by compliment...,non-suicide,"['weird', 'not', 'get', 'affect', 'compliment'...","['weird', 'not', 'get', 'affected', 'complimen...",weird not get affected compliment coming someo...,0.066667,0.000000,0.0,0.133333,0.133333,0.133333,0.133333,0.000000,0.066667,0.133333,0.200000
2,2,finally is almost over so i can never hear has...,non-suicide,"['final', 'almost', 'never', 'hear', 'bad', 'y...","['finally', 'almost', 'never', 'hear', 'bad', ...",finally almost never hear bad year ever swear ...,0.100000,0.100000,0.0,0.150000,0.050000,0.150000,0.100000,0.050000,0.100000,0.100000,0.100000
3,3,i need help just help me i am crying so hard,suicide,"['need', 'help', 'help', 'cri', 'hard']","['need', 'help', 'help', 'cry', 'hard']",need help help cry hard,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000
4,4,i m so lost hello my name is adam and i ve bee...,suicide,"['lost', 'hello', 'name', 'adam', 'struggl', '...","['lost', 'hello', 'name', 'adam', 'struggling'...",lost hello name adam struggling year afraid pa...,0.146226,0.103774,0.0,0.066038,0.037736,0.075472,0.198113,0.146226,0.061321,0.042453,0.122642


In [3]:
emotion_df.drop(columns = ["text","lemmatized_processed_text","class","stemmed_processed_text","Unnamed: 0"],axis=1,inplace=True)
emotion_df.head()

,clean_lemmatized_processed_text,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,anticipation
0,ex wife threatening suicide recently left wife...,0.125000,0.125000,0.0,0.078125,0.078125,0.078125,0.171875,0.093750,0.062500,0.078125,0.109375
1,weird not get affected compliment coming someo...,0.066667,0.000000,0.0,0.133333,0.133333,0.133333,0.133333,0.000000,0.066667,0.133333,0.200000
2,finally almost never hear bad year ever swear ...,0.100000,0.100000,0.0,0.150000,0.050000,0.150000,0.100000,0.050000,0.100000,0.100000,0.100000
3,need help help cry hard,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000
4,lost hello name adam struggling year afraid pa...,0.146226,0.103774,0.0,0.066038,0.037736,0.075472,0.198113,0.146226,0.061321,0.042453,0.122642


In [4]:
emotion_df_2 = pd.read_csv("lexicon_approach2.csv")
emotion_df_2.head()


,Unnamed: 0,text,class,stemmed_processed_text,lemmatized_processed_text,clean_lemmatized_processed_text,neg,neu,pos,compound
0,0,ex wife threatening suicide recently i left my...,suicide,"['ex', 'wife', 'threaten', 'suicid', 'recent',...","['ex', 'wife', 'threatening', 'suicide', 'rece...",ex wife threatening suicide recently left wife...,0.378,0.437,0.186,-0.9655
1,1,am i weird i do not get affected by compliment...,non-suicide,"['weird', 'not', 'get', 'affect', 'compliment'...","['weird', 'not', 'get', 'affected', 'complimen...",weird not get affected compliment coming someo...,0.225,0.529,0.245,0.0984
2,2,finally is almost over so i can never hear has...,non-suicide,"['final', 'almost', 'never', 'hear', 'bad', 'y...","['finally', 'almost', 'never', 'hear', 'bad', ...",finally almost never hear bad year ever swear ...,0.259,0.433,0.308,0.2025
3,3,i need help just help me i am crying so hard,suicide,"['need', 'help', 'help', 'cri', 'hard']","['need', 'help', 'help', 'cry', 'hard']",need help help cry hard,0.413,0.092,0.495,0.2263
4,4,i m so lost hello my name is adam and i ve bee...,suicide,"['lost', 'hello', 'name', 'adam', 'struggl', '...","['lost', 'hello', 'name', 'adam', 'struggling'...",lost hello name adam struggling year afraid pa...,0.368,0.505,0.127,-0.9965


In [5]:
emotion_df_2.drop(columns = ["text","lemmatized_processed_text","stemmed_processed_text","Unnamed: 0","clean_lemmatized_processed_text"],axis=1,inplace=True)
emotion_df_2.head()

,class,neg,neu,pos,compound
0,suicide,0.378,0.437,0.186,-0.9655
1,non-suicide,0.225,0.529,0.245,0.0984
2,non-suicide,0.259,0.433,0.308,0.2025
3,suicide,0.413,0.092,0.495,0.2263
4,suicide,0.368,0.505,0.127,-0.9965


In [6]:
####get POS_tagged
pos_df = pd.read_csv('pos_tagged_new_negation.csv')

pos_df.dropna(inplace=True)

pos_df['tagged_tokens'] = pos_df['tagged_tokens'].apply(lambda x: [str(i) for i in ast.literal_eval(x)])

pos_df.drop(columns = ["text","lemmatized_processed_text","tokens","class"], inplace = True)
pos_df["tagged_tokens"] = pos_df["tagged_tokens"].apply(lambda x: " ".join(x))
pos_df.head()

,tagged_tokens
0,ex_NN wife_NN threatening_VBG suicide_NN recen...
1,weird_JJ not_RB get_VB affected_VBN compliment...
2,finally_RB almost_RB never_RB hear_VB bad_JJ y...
3,need_NN help_NN help_NN cry_NN hard_RB
4,lost_VBN hello_UH name_NN adam_NN struggling_V...


In [7]:
combined_df = pd.concat([emotion_df, emotion_df_2,pos_df ], axis=1)
combined_df.dropna(inplace=True)

In [8]:
## Normalizing Data as naive bayes dont accept negative
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
combined_df['norm_compound'] = scaler.fit_transform(combined_df[['compound']])
combined_df.head()

,clean_lemmatized_processed_text,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,anticipation,class,neg,neu,pos,compound,tagged_tokens,norm_compound
0,ex wife threatening suicide recently left wife...,0.125000,0.125000,0.0,0.078125,0.078125,0.078125,0.171875,0.093750,0.062500,0.078125,0.109375,suicide,0.378,0.437,0.186,-0.9655,ex_NN wife_NN threatening_VBG suicide_NN recen...,0.01725
1,weird not get affected compliment coming someo...,0.066667,0.000000,0.0,0.133333,0.133333,0.133333,0.133333,0.000000,0.066667,0.133333,0.200000,non-suicide,0.225,0.529,0.245,0.0984,weird_JJ not_RB get_VB affected_VBN compliment...,0.54920
2,finally almost never hear bad year ever swear ...,0.100000,0.100000,0.0,0.150000,0.050000,0.150000,0.100000,0.050000,0.100000,0.100000,0.100000,non-suicide,0.259,0.433,0.308,0.2025,finally_RB almost_RB never_RB hear_VB bad_JJ y...,0.60125
3,need help help cry hard,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,suicide,0.413,0.092,0.495,0.2263,need_NN help_NN help_NN cry_NN hard_RB,0.61315
4,lost hello name adam struggling year afraid pa...,0.146226,0.103774,0.0,0.066038,0.037736,0.075472,0.198113,0.146226,0.061321,0.042453,0.122642,suicide,0.368,0.505,0.127,-0.9965,lost_VBN hello_UH name_NN adam_NN struggling_V...,0.00175


In [9]:
def get_metrics(y_test, y_pred):
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(conf_matrix)
    TP = conf_matrix[0][0]
    FN = conf_matrix[1][0]
    FP = conf_matrix[0][1]
    TN = conf_matrix[1][1]

    accuracy = accuracy_score(y_test, y_pred)*100
    precision = precision_score(y_test, y_pred)*100
    recall = recall_score(y_test, y_pred)*100

    print('TP:',TP); print('FN:',FN); print('FP:',FP) ;print('TN:',TN)
    print("Accuracy: ", accuracy)
    print('Precision:', precision)
    print('Recall:', recall)

    f1_score = 2*((precision * recall) / ((precision + recall)))
    print('F1 Score:', f1_score)

    f2score = ((1 + 2**2) * precision * recall) / (2**2 * precision + recall)
    print('F2 Score:', f2score)

### TESTING 3 VECTORS

#### NRC FLEX

In [10]:
##reading the domain dict from the text file
with open("domain_dict_postag_lem.txt", "r") as file:
    list_of_lem = [line.strip() for line in file]

In [11]:
tagged_tokens_list = []
for index, value in combined_df['tagged_tokens'].items():
    tagged_tokens_list.append(value)
len(tagged_tokens_list)

231979

In [12]:
tagged_tokens_list.extend(list_of_lem)
len(tagged_tokens_list)

231979

In [13]:
import scipy.sparse as sp

# Create a sparse matrix for the extra features
extra_features_sparse = sp.csr_matrix(combined_df[["fear", "anger", "anticip", "trust", "surprise", "positive", "negative", "sadness","disgust", "joy", "anticipation"]].values)

# Fit and transform the TF-IDF matrix
tfidf_vectorizer_sa_nrc = TfidfVectorizer()
tfidf_sa_nrc = tfidf_vectorizer_sa_nrc.fit_transform(tagged_tokens_list)

# Combine the two sparse matrices using hstack
all_features_sa_nrc = sp.hstack([tfidf_sa_nrc, extra_features_sparse])

In [14]:
train_X_sa_nrc, test_X_sa_nrc, train_y_sa_nrc, test_y_sa_nrc = train_test_split(all_features_sa_nrc,combined_df['class'],test_size=0.3, random_state = 42)

In [15]:
print("n_samples: %d, n_features: %d" % train_X_sa_nrc.shape)
print("n_samples: %d, n_features: %d" % test_X_sa_nrc.shape)

n_samples: 162385, n_features: 47588
n_samples: 69594, n_features: 47588


In [16]:
train_X_sa_nrc.shape

(162385, 47588)

In [17]:
test_y_sa_nrc

208365    non-suicide
49348         suicide
163403    non-suicide
116631    non-suicide
149784    non-suicide
             ...     
139580        suicide
128493    non-suicide
185478    non-suicide
214175    non-suicide
212553    non-suicide
Name: class, Length: 69594, dtype: object

#### VADER1  (Pos, neg, neu, comp)

In [18]:
extra_features_sparse = sp.csr_matrix(combined_df[["pos","neg","neu","norm_compound"]].values)

# Fit and transform the TF-IDF matrix
tfidf_vectorizer_sa_va_1 = TfidfVectorizer()
tfidf_sa_va_1 = tfidf_vectorizer_sa_va_1.fit_transform(tagged_tokens_list)

# Combine the two sparse matrices using hstack
all_features_sa_va_1 = sp.hstack([tfidf_sa_va_1, extra_features_sparse])

In [19]:
train_X_sa_va_1, test_X_sa_va_1, train_y_sa_va_1, test_y_sa_va_1 = train_test_split(all_features_sa_va_1,combined_df['class'],test_size=0.3, random_state = 42)

In [20]:
print("n_samples: %d, n_features: %d" % train_X_sa_va_1.shape)
print("n_samples: %d, n_features: %d" % test_X_sa_va_1.shape)

n_samples: 162385, n_features: 47581
n_samples: 69594, n_features: 47581


#### VADER2  (comp)

In [21]:
extra_features_sparse = sp.csr_matrix(combined_df[["norm_compound"]].values)

# Fit and transform the TF-IDF matrix
tfidf_vectorizer_sa_va_2 = TfidfVectorizer()
tfidf_sa_va_2 = tfidf_vectorizer_sa_va_2.fit_transform(tagged_tokens_list)

# Combine the two sparse matrices using hstack
all_features_sa_va_2 = sp.hstack([tfidf_sa_va_2, extra_features_sparse])

In [22]:
train_X_sa_va_2, test_X_sa_va_2, train_y_sa_va_2, test_y_sa_va_2 = train_test_split(all_features_sa_va_2,combined_df['class'],test_size=0.3, random_state = 42)

In [23]:
print("n_samples: %d, n_features: %d" % train_X_sa_va_2.shape)
print("n_samples: %d, n_features: %d" % test_X_sa_va_2.shape)

n_samples: 162385, n_features: 47578
n_samples: 69594, n_features: 47578


## Naive Bayes

#### NRC LEX

In [24]:
#Naive Bayes Classifier
naive_bayes_classifier_sa_nrc = MultinomialNB()
naive_bayes_classifier_sa_nrc.fit(train_X_sa_nrc, train_y_sa_nrc)
#predicted y
y_pred_nb_sa_nrc = naive_bayes_classifier_sa_nrc.predict(test_X_sa_nrc)

In [25]:
nb_accuracy_sa_nrc = accuracy_score(y_pred_nb_sa_nrc, test_y_sa_nrc)
print("Naive Bayes Accuracy Score -> ", nb_accuracy_sa_nrc * 100)

Naive Bayes Accuracy Score ->  89.66002816334742


In [26]:
print(metrics.classification_report(test_y_sa_nrc, y_pred_nb_sa_nrc, target_names=['Non-Suicide', 'Suicide'], digits=4))

              precision    recall  f1-score   support

 Non-Suicide     0.9449    0.8425    0.8908     34833
     Suicide     0.8577    0.9508    0.9018     34761

    accuracy                         0.8966     69594
   macro avg     0.9013    0.8967    0.8963     69594
weighted avg     0.9013    0.8966    0.8963     69594



In [27]:
conf_matrix = confusion_matrix(test_y_sa_nrc, y_pred_nb_sa_nrc)
print(conf_matrix)

[[29348  5485]
 [ 1711 33050]]


#### VADER1  (Pos, neg, neu, comp)

In [28]:
#Naive Bayes Classifier
naive_bayes_classifier_sa_va_1 = MultinomialNB()
naive_bayes_classifier_sa_va_1.fit(train_X_sa_va_1, train_y_sa_va_1)
#predicted y
y_pred_nb_sa_va_1 = naive_bayes_classifier_sa_va_1.predict(test_X_sa_va_1)

In [29]:
nb_accuracy_sa_va_1 = accuracy_score(y_pred_nb_sa_va_1, test_y_sa_va_1)
print("Naive Bayes Accuracy Score -> ", nb_accuracy_sa_va_1 * 100)

Naive Bayes Accuracy Score ->  90.4316464062994


In [30]:
recall = recall_score(test_y_sa_va_1, y_pred_nb_sa_va_1, average='macro')
print("Recall score:", recall)

Recall score: 0.9043568413229315


In [31]:
print(metrics.classification_report(test_y_sa_va_1, y_pred_nb_sa_va_1, target_names=['Non-Suicide', 'Suicide'], digits=4))

              precision    recall  f1-score   support

 Non-Suicide     0.9387    0.8653    0.9005     34833
     Suicide     0.8749    0.9434    0.9078     34761

    accuracy                         0.9043     69594
   macro avg     0.9068    0.9044    0.9042     69594
weighted avg     0.9068    0.9043    0.9042     69594



In [32]:
conf_matrix = confusion_matrix(test_y_sa_va_1, y_pred_nb_sa_va_1)
print(conf_matrix)

[[30142  4691]
 [ 1968 32793]]


#### VADER1  (comp)

In [33]:
#Naive Bayes Classifier
naive_bayes_classifier_sa_va_2 = MultinomialNB()
naive_bayes_classifier_sa_va_2.fit(train_X_sa_va_2, train_y_sa_va_2)
#predicted y
y_pred_nb_sa_va_2 = naive_bayes_classifier_sa_va_2.predict(test_X_sa_va_2)

In [34]:
nb_accuracy_sa_va_2 = accuracy_score(y_pred_nb_sa_va_2, test_y_sa_va_2)
print("Naive Bayes Accuracy Score -> ", nb_accuracy_sa_va_2 * 100)

Naive Bayes Accuracy Score ->  90.05086645400465


In [35]:
print(metrics.classification_report(test_y_sa_va_2, y_pred_nb_sa_va_2, target_names=['Non-Suicide', 'Suicide'], digits=4))

              precision    recall  f1-score   support

 Non-Suicide     0.9416    0.8542    0.8958     34833
     Suicide     0.8663    0.9470    0.9048     34761

    accuracy                         0.9005     69594
   macro avg     0.9040    0.9006    0.9003     69594
weighted avg     0.9040    0.9005    0.9003     69594



## Logistic Regression

#### NRC LEX

In [36]:
# fit the training dataset on the Logistic Regression classifier
logreg_sa_nrc = LogisticRegression(max_iter=200)
logreg_sa_nrc.fit(train_X_sa_nrc,train_y_sa_nrc)
# predict the labels on validation dataset
y_pred_logreg_sa_nrc = logreg_sa_nrc.predict(test_X_sa_nrc)

In [37]:
logreg_accuracy_sa_nrc = accuracy_score(y_pred_logreg_sa_nrc, test_y_sa_nrc)
print("Logistic Regression Accuracy Score -> ", logreg_accuracy_sa_nrc * 100)

Logistic Regression Accuracy Score ->  93.56553725895911


In [38]:
print(metrics.classification_report(test_y_sa_nrc, y_pred_logreg_sa_nrc, target_names=['Non-Suicide', 'Suicide'], digits=4))

              precision    recall  f1-score   support

 Non-Suicide     0.9276    0.9452    0.9363     34833
     Suicide     0.9440    0.9261    0.9350     34761

    accuracy                         0.9357     69594
   macro avg     0.9358    0.9356    0.9356     69594
weighted avg     0.9358    0.9357    0.9356     69594



In [39]:
conf_matrix = confusion_matrix(test_y_sa_nrc, y_pred_logreg_sa_nrc)
print(conf_matrix)

[[32924  1909]
 [ 2569 32192]]


#### VADER1  (Pos, neg, neu, comp)

In [40]:
# fit the training dataset on the Logistic Regression classifier
logreg_sa_va_1 = LogisticRegression(max_iter=1000)
logreg_sa_va_1.fit(train_X_sa_va_1,train_y_sa_va_1)
# predict the labels on validation dataset
y_pred_logreg_sa_va_1 = logreg_sa_va_1.predict(test_X_sa_va_1)

In [41]:
logreg_accuracy_sa_va_1 = accuracy_score(y_pred_logreg_sa_va_1, test_y_sa_va_1)
print("Logistic Regression Accuracy Score -> ", logreg_accuracy_sa_va_1 * 100)

Logistic Regression Accuracy Score ->  93.60002299048769


In [42]:
print(metrics.classification_report(test_y_sa_va_1, y_pred_logreg_sa_va_1, target_names=['Non-Suicide', 'Suicide'], digits=4))

              precision    recall  f1-score   support

 Non-Suicide     0.9281    0.9454    0.9367     34833
     Suicide     0.9443    0.9266    0.9353     34761

    accuracy                         0.9360     69594
   macro avg     0.9362    0.9360    0.9360     69594
weighted avg     0.9362    0.9360    0.9360     69594



In [43]:
conf_matrix = confusion_matrix(test_y_sa_va_1, y_pred_logreg_sa_va_1)
print(conf_matrix)

[[32932  1901]
 [ 2553 32208]]


#### VADER1  (comp)

In [44]:
# fit the training dataset on the Logistic Regression classifier
logreg_sa_va_2 = LogisticRegression(max_iter=200)
logreg_sa_va_2.fit(train_X_sa_va_2,train_y_sa_va_2)
# predict the labels on validation dataset
y_pred_logreg_sa_va_2 = logreg_sa_va_2.predict(test_X_sa_va_2)

In [45]:
logreg_accuracy_sa_va_2 = accuracy_score(y_pred_logreg_sa_va_2, test_y_sa_va_2)
print("Logistic Regression Accuracy Score -> ", logreg_accuracy_sa_va_2 * 100)

Logistic Regression Accuracy Score ->  93.57128488088054


In [46]:
print(metrics.classification_report(test_y_sa_va_2, y_pred_logreg_sa_va_2, target_names=['Non-Suicide', 'Suicide'], digits=4))

              precision    recall  f1-score   support

 Non-Suicide     0.9270    0.9461    0.9364     34833
     Suicide     0.9449    0.9253    0.9350     34761

    accuracy                         0.9357     69594
   macro avg     0.9359    0.9357    0.9357     69594
weighted avg     0.9359    0.9357    0.9357     69594



## SVM

#### NRC FLEX

In [47]:
# fit the training dataset on the SVM classifier
linearSVC_sa_nrc = LinearSVC()
linearSVC_sa_nrc.fit(train_X_sa_nrc,train_y_sa_nrc)
# predict the labels on validation dataset
y_pred_SVC_sa_nrc = linearSVC_sa_nrc.predict(test_X_sa_nrc)


In [48]:
svc_accuracy_sa_nrc = accuracy_score(y_pred_SVC_sa_nrc, test_y_sa_nrc)
print("SVM Accuracy Score -> ",svc_accuracy_sa_nrc*100)

SVM Accuracy Score ->  93.5080610397448


In [49]:
print(metrics.classification_report(test_y_sa_nrc, y_pred_SVC_sa_nrc, target_names=['Non-Suicide', 'Suicide'], digits=4))

              precision    recall  f1-score   support

 Non-Suicide     0.9286    0.9428    0.9356     34833
     Suicide     0.9418    0.9273    0.9345     34761

    accuracy                         0.9351     69594
   macro avg     0.9352    0.9351    0.9351     69594
weighted avg     0.9352    0.9351    0.9351     69594



In [50]:
conf_matrix = confusion_matrix(test_y_sa_nrc, y_pred_SVC_sa_nrc)
print(conf_matrix)

[[32842  1991]
 [ 2527 32234]]


#### VADER1  (Pos, neg, neu, comp)

In [51]:
# fit the training dataset on the SVM classifier
linearSVC_sa_va_1 = LinearSVC()
linearSVC_sa_va_1.fit(train_X_sa_va_1,train_y_sa_va_1)
# predict the labels on validation dataset
y_pred_SVC_sa_va_1 = linearSVC_sa_va_1.predict(test_X_sa_va_1)


In [52]:
svc_accuracy_sa_va_1 = accuracy_score(y_pred_SVC_sa_va_1, test_y_sa_va_1)
print("SVM Accuracy Score -> ",svc_accuracy_sa_va_1*100)

SVM Accuracy Score ->  93.53823605483231


In [53]:
print(metrics.classification_report(test_y_sa_va_1, y_pred_SVC_sa_va_1, target_names=['Non-Suicide', 'Suicide'], digits=4))

              precision    recall  f1-score   support

 Non-Suicide     0.9293    0.9426    0.9359     34833
     Suicide     0.9417    0.9281    0.9348     34761

    accuracy                         0.9354     69594
   macro avg     0.9355    0.9354    0.9354     69594
weighted avg     0.9355    0.9354    0.9354     69594



In [54]:
conf_matrix = confusion_matrix(test_y_sa_va_1, y_pred_SVC_sa_va_1)
print(conf_matrix)

[[32835  1998]
 [ 2499 32262]]


#### VADER1  (comp)

In [55]:
# fit the training dataset on the SVM classifier
linearSVC_sa_va_2 = LinearSVC()
linearSVC_sa_va_2.fit(train_X_sa_va_2,train_y_sa_va_2)
# predict the labels on validation dataset
y_pred_SVC_sa_va_2 = linearSVC_sa_va_2.predict(test_X_sa_va_2)


In [56]:
svc_accuracy_sa_va_2 = accuracy_score(y_pred_SVC_sa_va_2, test_y_sa_va_2)
print("SVM Accuracy Score -> ",svc_accuracy_sa_va_2*100)

SVM Accuracy Score ->  93.53679914935196


In [57]:
print(metrics.classification_report(test_y_sa_va_2, y_pred_SVC_sa_va_2, target_names=['Non-Suicide', 'Suicide'], digits=4))

              precision    recall  f1-score   support

 Non-Suicide     0.9289    0.9431    0.9359     34833
     Suicide     0.9421    0.9276    0.9348     34761

    accuracy                         0.9354     69594
   macro avg     0.9355    0.9354    0.9354     69594
weighted avg     0.9355    0.9354    0.9354     69594

